A notebook written in TensorFlow 2.0 designed to use a Pre-trained classifer from TensorFlow Hub to classify pictures of flowers. After running a the model, the model is deployed to TensorFlow Lite

## Set up libraries

In [0]:
from __future__ import absolute_import, division, print_function

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import matplotlib.pyplot as plt


import itertools
import os

import numpy as np
import tensorflow_hub as hub

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

TensorFlow 2.x selected.
Version:  2.0.0
Eager mode:  True
Hub version:  0.7.0
GPU is available


In [0]:
class ResetStatesCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

## Download TF Hub Module

Use one of the selections provided.

In [0]:
module_selection = ("inception_v3", 299) #@param ["(\"mobilenet_v2_035_224\", 224)", "(\"inception_v3\", 299)", "(\"inception_resnet_v2\", 299)"] {type:"raw", allow-input: true}
handle_base, pixels = module_selection
MODULE_HANDLE ="https://tfhub.dev/google/imagenet/{}/classification/4".format(handle_base)
IMAGE_SIZE = (pixels, pixels)
print("Using {} with input size {}".format(MODULE_HANDLE, IMAGE_SIZE))

BATCH_SIZE = 32 #@param {type:"integer"}

Using https://tfhub.dev/google/imagenet/inception_v3/classification/4 with input size (299, 299)


### Download the Dataset

Input URL.
Input directory name.

In [0]:
URL = 'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz' #@param {type:"string"}
dir_name = 'flower_photos' #@param {type:"string"}

data_dir = tf.keras.utils.get_file(dir_name,
                                   URL,
                                   extract=True)

### *******Understand the data

List the directories with the following terminal command

In [0]:
data_dir_base = os.path.dirname(data_dir)
!find $data_dir_base -type d -print

/root/.keras/datasets
/root/.keras/datasets/flower_photos
/root/.keras/datasets/flower_photos/roses
/root/.keras/datasets/flower_photos/tulips
/root/.keras/datasets/flower_photos/dandelion
/root/.keras/datasets/flower_photos/daisy
/root/.keras/datasets/flower_photos/sunflowers


### Data Preparation and Augmentation

In [0]:
datagen_kwargs = dict(rescale=1./255, validation_split=.20)
dataflow_kwargs = dict(target_size=IMAGE_SIZE, 
                       batch_size=BATCH_SIZE)

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                          **datagen_kwargs)

valid_generator = valid_datagen.flow_from_directory(
                          data_dir, 
                          subset="validation", 
                          shuffle=False, 
                          **dataflow_kwargs)

do_data_augmentation = True #@param {type:"boolean"}
if do_data_augmentation:
  train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      rotation_range=40,
      horizontal_flip=True,
      width_shift_range=0.2, 
      height_shift_range=0.2,
      shear_range=0.2, 
      zoom_range=0.2,
      **datagen_kwargs)
else:
  train_datagen = valid_datagen
  
train_generator = train_datagen.flow_from_directory(
                            data_dir, 
                            subset="training", 
                            shuffle=True, 
                            **dataflow_kwargs)

Found 731 images belonging to 5 classes.
Found 2939 images belonging to 5 classes.


The resulting object is an iterator that returns image_batch, label_batch pairs.

In [0]:
for image_batch, label_batch in train_generator:
  print("Image batch shape: ", image_batch.shape)
  print("Label batch shape: ", label_batch.shape)
  break

## Create and train the model


### Define the learning model

For speed, start out with a non-trainable `feature_extractor_layer`, enable fine-tuning for greater accuracy.

In [0]:
do_fine_tuning = False #@param {type:"boolean"}

In [0]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

print("Building model with", MODULE_HANDLE)
model = tf.keras.Sequential([
    hub.KerasLayer(MODULE_HANDLE, trainable=do_fine_tuning),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax',
                          kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])
model.build((None,)+IMAGE_SIZE+(3,))
model.summary()


lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-4 * 10**(epoch / 30))
reset_states = ResetStatesCallback()
optimizer = tf.keras.optimizers.Adam(lr=1e-4)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])


steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = valid_generator.samples // valid_generator.batch_size

early_stopping = tf.keras.callbacks.EarlyStopping(patience=10)

hist = model.fit_generator(
            train_generator,
            epochs=100, 
            callbacks=[early_stopping, lr_schedule, reset_states],
            steps_per_epoch=steps_per_epoch,
            validation_data=valid_generator,
            validation_steps=validation_steps).history

In [0]:
plt.semilogx(hist["lr"], hist["loss"])
plt.axis([1e-4, 1e-1, 0, 1.5])

### Train the updated Model

In [0]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

print("Building model with", MODULE_HANDLE)
model = tf.keras.Sequential([
    hub.KerasLayer(MODULE_HANDLE, trainable=do_fine_tuning),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax',
                          kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])
model.build((None,)+IMAGE_SIZE+(3,))
model.summary()

reset_states = ResetStatesCallback()
optimizer = tf.keras.optimizers.Adam(lr=1e-3)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])


steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = valid_generator.samples // valid_generator.batch_size

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "my_checkpoint.h5", save_best_only=True)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10)

hist = model.fit_generator(
            train_generator,
            epochs=100, 
            callbacks=[early_stopping, model_checkpoint, reset_states],
            steps_per_epoch=steps_per_epoch,
            validation_data=valid_generator,
            validation_steps=validation_steps).history

MobileNet - acc: 0.856 with aug + dropout no FT

Inception - acc: 0.911 with aug, with/without dropout, no FT, lr=le-3

In [0]:
model = tf.keras.models.load_model("my_checkpoint.h5")

## Visualisation

In [0]:
import matplotlib.pyplot as plt

# Retrieve a list of accuracy results on training and test datasets for each training epoch
acc = hist['acc']
val_acc = hist['val_acc']

# Retrieve a list of list results on training and test datasets for each training epoch
loss = hist['loss']
val_loss = hist['val_loss']


# Get number of epochs
epochs = range(len(acc))

# Plot training and validation accuracy per epoch
plt.plot(epochs, acc, label='Training acc')
plt.plot(epochs, val_acc, label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

# Plot training and validation loss per epoch
plt.plot(epochs, loss, label='Training loss')
plt.plot(epochs, val_loss, label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

## Check the predictions
Get the ordered list of class names

In [0]:
class_names = sorted(train_generator.class_indices.items(), key=lambda pair:pair[1])
class_names = np.array([key.title() for key, value in class_names])
class_names

Run the image batch through the model and convert the indices to class names.

In [0]:
predicted_batch = model.predict(image_batch)
predicted_id = np.argmax(predicted_batch, axis=-1)
predicted_label_batch = class_names[predicted_id]

Plot the results

In [0]:
label_id = np.argmax(label_batch, axis=-1)

plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  color = "green" if predicted_id[n] == label_id[n] else "red"
  plt.title(predicted_label_batch[n].title(), color=color)
  plt.axis('off')
_ = plt.suptitle("Model predictions (green: correct, red: incorrect)")

Generate a saved model for deployment to TF Serving or TF Lite (on mobile) as follows.

In [0]:
saved_model_path = "/saved_model/1"
tf.saved_model.save(model, saved_model_path)

## Deployment to TensorFlow Lite

Optimise the model and convert the trained model to TF Lite and apply post-training tools from the [TensorFlow Model Optimization Toolkit](https://www.tensorflow.org/model_optimization). 

  * Converting without optimization provides the same results as before (up to roundoff error).
  * Converting with optimization without any data quantizes the model weights to 8 bits, but inference still uses floating-point computation for the neural network activations. This reduces model size almost by a factor of 4 and improves CPU latency on mobile devices.
  * On top, computation of the neural network activations can be quantized to 8-bit integers as well if a small reference dataset is provided to calibrate the quantization range. On a mobile device, this accelerates inference further and makes it possible to run on accelerators like EdgeTPU.

In [0]:
#@title Optimization settings
optimize_lite_model = True  #@param {type:"boolean"}
#@markdown Setting a value greater than zero enables quantization of neural network activations. A few dozen is already a useful amount.
num_calibration_examples = 154  #@param {type:"slider", min:0, max:1000, step:1}
representative_dataset = None
if optimize_lite_model and num_calibration_examples:
  # Use a bounded number of training examples without labels for calibration.
  # TFLiteConverter expects a list of input tensors, each with batch size 1.
  representative_dataset = lambda: itertools.islice(
      ([image[None, ...]] for batch, _ in train_generator for image in batch),
      num_calibration_examples)

#@markdown Select mode of optimization
mode = "Speed" #@param ["Default", "Storage", "Speed"]

if mode == 'Storage':
  optimization = tf.lite.Optimize.OPTIMIZE_FOR_SIZE
elif mode == 'Speed':
  optimization = tf.lite.Optimize.OPTIMIZE_FOR_LATENCY
else:
  optimization = tf.lite.Optimize.DEFAULT


Convert the model

In [0]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)
if optimize_lite_model:
  converter.optimizations = [optimization]
  if representative_dataset:  # This is optional, see above.
    converter.representative_dataset = representative_dataset
tflite_model = converter.convert()

tflite_model_file = 'model.tflite'

with open(tflite_model_file, "wb") as f:
  f.write(tflite_model)
print("Wrote %sTFLite model of %d bytes." %
      ("optimized " if optimize_lite_model else "", len(tflite_model)))

Test the model with the TF Lite Interpreter to examine the resulting quality

In [0]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
# This little helper wraps the TF Lite interpreter as a numpy-to-numpy function.
def lite_model(images):
  interpreter.allocate_tensors()
  interpreter.set_tensor(interpreter.get_input_details()[0]['index'], images)
  interpreter.invoke()
  return interpreter.get_tensor(interpreter.get_output_details()[0]['index'])

In [0]:
#@markdown For rapid experimentation, start with a moderate number of examples.
num_eval_examples = 57  #@param {type:"slider", min:0, max:700}
eval_dataset = ((image, label)  # TFLite expects batch size 1.
                for batch in train_generator
                for (image, label) in zip(*batch))
count = 0
count_lite_tf_agree = 0
count_lite_correct = 0
for image, label in eval_dataset:
  probs_lite = lite_model(image[None, ...])[0]
  probs_tf = model(image[None, ...]).numpy()[0]
  y_lite = np.argmax(probs_lite)
  y_tf = np.argmax(probs_tf)
  y_true = np.argmax(label)
  count +=1
  if y_lite == y_tf: count_lite_tf_agree += 1
  if y_lite == y_true: count_lite_correct += 1
  if count >= num_eval_examples: break
print("TF Lite model agrees with original model on %d of %d examples (%g%%)." %
      (count_lite_tf_agree, count, 100.0 * count_lite_tf_agree / count))
print("TF Lite model is accurate on %d of %d examples (%g%%)." %
      (count_lite_correct, count, 100.0 * count_lite_correct / count))

Download the file

In [0]:
try:
  from google.colab import files

  files.download(tflite_model_file)
except:
  pass